In [16]:
import pandas as pd
import time

In [2]:
all_games = pd.read_html("https://www.pro-football-reference.com/boxscores/game-scores.htm")

In [7]:
all_games_df = pd.DataFrame(all_games[0])

In [8]:
all_games_df

,Rk,Score,PtsW,PtsL,PtTot,PD,Count,Unnamed: 7,Last Game
0,1,20-17,20,17,37,3,272,all games,"Kansas City Chiefs vs. New York Giants, Novemb..."
1,2,27-24,27,24,51,3,222,all games,"Los Angeles Chargers vs. Philadelphia Eagles, ..."
2,3,17-14,17,14,31,3,198,all games,"Cleveland Browns vs. Denver Broncos, October 2..."
3,4,23-20,23,20,43,3,191,all games,"Jacksonville Jaguars vs. Miami Dolphins, Octob..."
4,5,24-17,24,17,41,7,169,all games,"Tennessee Titans vs. Chicago Bears, November 8..."
...,...,...,...,...,...,...,...,...,...
1064,1065,59-16,59,16,75,43,1,all games,"St. Louis Rams vs. Atlanta Falcons, November 1..."
1065,1066,57-31,57,31,88,26,1,all games,"St. Louis Rams vs. San Diego Chargers, October..."
1066,1067,54-51,54,51,105,3,1,all games,"Los Angeles Rams vs. Kansas City Chiefs, Novem..."
1067,1068,39-27,39,27,66,12,1,all games,"Baltimore Ravens vs. Indianapolis Colts, Decem..."


In [12]:
WinScores = all_games_df["PtsW"].tolist()
LossScores = all_games_df["PtsL"].tolist()

In [45]:
#out_df = pd.DataFrame(columns=['Rk', 'Week', 'Day', 'Date', 'Unnamed: 4', 'Winner/tie', 'Unnamed: 6',
#       'Loser/tie', 'Unnamed: 8', 'PtsW', 'PtsL', 'YdsW', 'TOW', 'YdsL',
#       'TOL'])
counter = 1
for W, L in zip(WinScores[991:], LossScores[991:]):
    temp_df = pd.read_html(f"https://www.pro-football-reference.com/boxscores/game_scores_find.cgi?pts_win={W}&pts_lose={L}")[0]
    out_df = out_df.append(temp_df)
    num_rows = len(out_df)
    print(f"Added: {W} - {L}, Number: {counter}, No_Lines: {num_rows}")
    counter += 1
    time.sleep(0.5)

Added: 19 - 19, Number: 1, No_Lines: 16869
Added: 42 - 40, Number: 2, No_Lines: 16870
Added: 52 - 13, Number: 3, No_Lines: 16871
Added: 24 - 8, Number: 4, No_Lines: 16872
Added: 64 - 0, Number: 5, No_Lines: 16873
Added: 44 - 42, Number: 6, No_Lines: 16874
Added: 59 - 28, Number: 7, No_Lines: 16875
Added: 55 - 27, Number: 8, No_Lines: 16876
Added: 57 - 33, Number: 9, No_Lines: 16877
Added: 43 - 18, Number: 10, No_Lines: 16878
Added: 43 - 43, Number: 11, No_Lines: 16879
Added: 41 - 26, Number: 12, No_Lines: 16880
Added: 44 - 37, Number: 13, No_Lines: 16881
Added: 65 - 24, Number: 14, No_Lines: 16882
Added: 48 - 35, Number: 15, No_Lines: 16883
Added: 46 - 14, Number: 16, No_Lines: 16884
Added: 41 - 39, Number: 17, No_Lines: 16885
Added: 66 - 0, Number: 18, No_Lines: 16886
Added: 45 - 23, Number: 19, No_Lines: 16887
Added: 39 - 29, Number: 20, No_Lines: 16888
Added: 36 - 18, Number: 21, No_Lines: 16889
Added: 39 - 30, Number: 22, No_Lines: 16890
Added: 21 - 8, Number: 23, No_Lines: 16891
A

In [68]:
out_df = out_df.rename(columns={"Unnamed: 6": "AtHome"})
out_df.to_csv("historic_match_scores.csv")

In [80]:
out_df["Date"] = out_df["Date"].astype("datetime64") 

In [50]:
# PtsW - Points scored by the winning team
# PtsL - Points score by the losing team
# YdsW - Yards gained by the winning team
# YdsL - Yards gained by the losing team
# TOW - Turnovers by winning Team
# TOL - Turnovers by losing Team

In [75]:
def get_home_team(Winner, Loser, AtHome):
    if AtHome == "@":
        HomeTeam = Loser
    else:
        HomeTeam = Winner
    
    return HomeTeam

In [85]:
out_df["HomeTeam"] = out_df.apply(lambda row : get_home_team(row["Winner/tie"],
                                  row["Loser/tie"], row["AtHome"]), axis = 1)

In [83]:
def get_season(DateField):
    """This function parses the given match date to process and the return the year of the season being played.

    Args:
        DateField (datetime): The column/field that contains the date that the match was played upon

    Returns:
        Int: The year of the season being played
    """
    if (DateField.month >= 1) & (DateField.month <= 3):
        SeasonYear = DateField.year -1
    else:
        SeasonYear = DateField.year
    return SeasonYear

In [86]:
out_df["Season"] = out_df.apply(lambda row : get_season(row["Date"]), axis = 1)

### Wild Card
* Added in 1978 (March), being the 1977 season

## 2020 Season

In [97]:
df_2020 = out_df[out_df["Season"] == 2020]
df_2020["Week"] = [row.lstrip().rstrip() for row in df_2020["Week"].astype("str")]

/Users/alessiomorteo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [98]:
print(f"Number of games played: {len(df_2020)}")

Number of games played: 269


In [104]:
df_2020.groupby(["Week"]).size().reset_index().rename(columns={0:'count'}).sort_values(by="count", ascending=False)

,Week,count
0,1,16
6,15,16
9,2,16
8,17,16
7,16,16
10,3,16
5,14,16
3,12,16
4,13,15
11,4,15


### Week Types

In [109]:
unique_weeks = {}
for season in out_df["Season"].unique():
    unique_weeks[season] = {}
    temp_df = out_df[out_df["Season"] == season]
    weeks_found = temp_df["Week"].unique().tolist()
    unique_weeks[season] = [str(i) for i in weeks_found]

In [112]:
all_time_unique = sorted({x for v in unique_weeks.values() for x in v})

In [114]:
out_df["Week"] = out_df["Week"].replace("Conf. Champ", "Champ")

## Max Month

In [127]:
import re

In [140]:
max_week = {}
for season in out_df["Season"].unique():
    max_week[season] = {}
    temp_df = out_df["Week"][out_df["Season"] == season].astype("str")
    week_values = [int(i) for i in temp_df if not re.search('[a-zA-Z]', i)]
    #week_values = [int(i) for i in week_values]
    max_week[season] = max(week_values)

In [106]:
def assign_non_standard_game_number(data, season):
    """Assign corrected week number to post-league games.

    Args:
        data (DataFrame): A dataframe containing the row for one league year only.
    """
    temp_df = data[data["Season" == season]]
    num_weeks = unique_weeks[season]
    highest_week = max_week[season] 
    temp_df["OriginalWeek"] = temp_df["Week"].copy()
    has_wild_card = 1

    non_standard_df = data[~data["OriginalWeek"] in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]]

    for i, j in enumerate(data["Week"]):
        if j == "Wild Card":
            has_wild_card 
            return final_week + post_game_max
        if row["Week"] == "Wild Card":
            new_week = final_week + post_game_max

,Rk,Week,Day,Date,Unnamed: 4,Winner/tie,AtHome,Loser/tie,Unnamed: 8,PtsW,PtsL,YdsW,TOW,YdsL,TOL,HomeTeam,Season
216,217,Wild Card,Sat,2021-01-09,W,Buffalo Bills,NaN,Indianapolis Colts,boxscore,27,24,397,0,472,0,Buffalo Bills,2020
112,113,Wild Card,Sun,2021-01-10,W,Baltimore Ravens,@,Tennessee Titans,boxscore,20,13,401,1,209,1,Tennessee Titans,2020
46,47,Wild Card,Sat,2021-01-09,W,Los Angeles Rams,@,Seattle Seahawks,boxscore,30,20,333,0,278,2,Seattle Seahawks,2020
32,33,Wild Card,Sat,2021-01-09,W,Tampa Bay Buccaneers,@,Washington Football Team,boxscore,31,23,507,1,375,1,Washington Football Team,2020
15,16,Wild Card,Sun,2021-01-10,W,New Orleans Saints,NaN,Chicago Bears,boxscore,21,9,385,1,239,0,New Orleans Saints,2020
0,1,Wild Card,Sun,2021-01-10,W,Cleveland Browns,@,Pittsburgh Steelers,boxscore,48,37,390,0,553,5,Pittsburgh Steelers,2020


## Super Bowl History